## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])

In [20]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 1500
application_types_to_replace = list(application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     5720
T4        1542
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C2400        6
C1720        6
C1257        5
C1600        5
C1260        3
C0           3
C2710        3
C1267        2
C3200        2
C1246        2
C1256        2
C1234        2
C4500        1
C8210        1
C2190        1
C2170        1
C4200        1
C1900        1
C1728        1
C2561        1
C3700        1
C6100        1
C2150        1
C1236        1
C1283        1
C1820        1
C1248        1
C2600        1
C1732        1
C2380        1
C1580        1
C5200        1
C1370        1
C2570     

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
modified_type_counts = classification_counts[classification_counts>1]
modified_type_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C2400        6
C1720        6
C1257        5
C1600        5
C1260        3
C0           3
C2710        3
C1267        2
C3200        2
C1246        2
C1256        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 300
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_variables = application_df.dtypes
categorical_variables

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [27]:
application_dummy_df = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION",
                                                     "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"]])
application_dummy_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [28]:
#combine dataframes
merged_df = application_df.merge(application_dummy_df, left_index = True, right_index = True)
merged_df = merged_df.drop(["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "SPECIAL_CONSIDERATIONS", "INCOME_AMT"], axis=1)
merged_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [12]:
merged_df.shape[1]

39

In [13]:
# Split our preprocessed data into our features and target arrays
y = merged_df["IS_SUCCESSFUL"].values
X = merged_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=38))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="elu"))

#Third hidden layer
nn.add(tf.keras.layers.Dense(units = 30, activation = "tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 30)                1170      
_________________________________________________________________
dense_10 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_11 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 31        
Total params: 3,061
Trainable params: 3,061
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss ="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [33]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs = 400)

Train on 25724 samples
Epoch 1/400
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5877 - accuracy: 0.7071
Epoch 2/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5686 - accuracy: 0.7210
Epoch 3/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5654 - accuracy: 0.7220
Epoch 4/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5638 - accuracy: 0.7219
Epoch 5/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5625 - accuracy: 0.7227
Epoch 6/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5623 - accuracy: 0.7226
Epoch 7/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5610 - accuracy: 0.7248
Epoch 8/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5604 - accuracy: 0.7254
Epoch 9/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5596 - accuracy: 0.7266
Epoch

Epoch 76/400
25724/25724 [==============================] - 2s 78us/sample - loss: 0.5507 - accuracy: 0.7310
Epoch 77/400
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5507 - accuracy: 0.7314
Epoch 78/400
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5507 - accuracy: 0.7308
Epoch 79/400
25724/25724 [==============================] - 2s 75us/sample - loss: 0.5508 - accuracy: 0.7313
Epoch 80/400
25724/25724 [==============================] - 4s 149us/sample - loss: 0.5514 - accuracy: 0.7306
Epoch 81/400
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5503 - accuracy: 0.7311
Epoch 82/400
25724/25724 [==============================] - 2s 83us/sample - loss: 0.5501 - accuracy: 0.7314
Epoch 83/400
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5506 - accuracy: 0.7303
Epoch 84/400
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5504 - accuracy: 0.7304
Epoch 85/400
2572

25724/25724 [==============================] - 1s 32us/sample - loss: 0.5492 - accuracy: 0.7315
Epoch 151/400
25724/25724 [==============================] - 1s 34us/sample - loss: 0.5489 - accuracy: 0.7324
Epoch 152/400
25724/25724 [==============================] - 2s 73us/sample - loss: 0.5489 - accuracy: 0.7318
Epoch 153/400
25724/25724 [==============================] - 4s 166us/sample - loss: 0.5493 - accuracy: 0.7316
Epoch 154/400
25724/25724 [==============================] - 10s 400us/sample - loss: 0.5492 - accuracy: 0.7316
Epoch 155/400
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5496 - accuracy: 0.7321
Epoch 156/400
25724/25724 [==============================] - 1s 33us/sample - loss: 0.5492 - accuracy: 0.7321
Epoch 157/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5489 - accuracy: 0.7325
Epoch 158/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5491 - accuracy: 0.7316
Epoch 159/400
25724/

25724/25724 [==============================] - 9s 340us/sample - loss: 0.5485 - accuracy: 0.7325
Epoch 225/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5498 - accuracy: 0.7314
Epoch 226/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5494 - accuracy: 0.7323
Epoch 227/400
25724/25724 [==============================] - 1s 32us/sample - loss: 0.5493 - accuracy: 0.7321
Epoch 228/400
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5491 - accuracy: 0.7320
Epoch 229/400
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5499 - accuracy: 0.7322
Epoch 230/400
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5493 - accuracy: 0.7317
Epoch 231/400
25724/25724 [==============================] - 8s 316us/sample - loss: 0.5490 - accuracy: 0.7315
Epoch 232/400
25724/25724 [==============================] - 11s 425us/sample - loss: 0.5485 - accuracy: 0.7328
Epoch 233/400
25724/

25724/25724 [==============================] - 1s 51us/sample - loss: 0.5481 - accuracy: 0.7326
Epoch 299/400
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5482 - accuracy: 0.7322
Epoch 300/400
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5481 - accuracy: 0.7332
Epoch 301/400
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5478 - accuracy: 0.7326
Epoch 302/400
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5483 - accuracy: 0.7328
Epoch 303/400
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5481 - accuracy: 0.7331
Epoch 304/400
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5484 - accuracy: 0.7325
Epoch 305/400
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5483 - accuracy: 0.7331
Epoch 306/400
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5483 - accuracy: 0.7318
Epoch 307/400
25724/2572

25724/25724 [==============================] - 2s 64us/sample - loss: 0.5476 - accuracy: 0.7320
Epoch 373/400
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5484 - accuracy: 0.7330
Epoch 374/400
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5475 - accuracy: 0.7314
Epoch 375/400
25724/25724 [==============================] - 2s 64us/sample - loss: 0.5476 - accuracy: 0.7326
Epoch 376/400
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5483 - accuracy: 0.7313
Epoch 377/400
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5477 - accuracy: 0.7317
Epoch 378/400
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5481 - accuracy: 0.7312
Epoch 379/400
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5481 - accuracy: 0.7313
Epoch 380/400
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5479 - accuracy: 0.7332
Epoch 381/400
25724/2572

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.5274 - accuracy: 0.7223
Loss: 0.5677968684021307, Accuracy: 0.7223323583602905


In [35]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_2.h5")